In [1]:
import tiktoken

In [2]:
#initialize byte pair encoding

tokenizer=tiktoken.get_encoding('gpt2')


In [3]:
#text to be encoded
text=(
    'hello do you like coffee?<|endoftext|> yes i like'
)
#call encode method which return ids of subword token
integers=tokenizer.encode(text,allowed_special={'<|endoftext|>'})
print(integers)

[31373, 466, 345, 588, 6891, 30, 50256, 3763, 1312, 588]


In [9]:
#now convert back token id back to text or decode

strings=tokenizer.decode(integers)
print(strings)

hello do you like coffee?<|endoftext|> yes i like
